# SQLAlchemy Practice

In [1]:
# SQLAlchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String # datatypes
from sqlalchemy import ForeignKey

from sqlalchemy import and_, or_, asc, desc, between # conjunctions
from sqlalchemy import text # for textual statements

from sqlalchemy.sql import select 
from sqlalchemy.sql import alias # represents 'as' in sql
from sqlalchemy.sql import func # standard sql functions

# Misc
import os

**SQLAlchemy** is a Python toolkit for dealing with databases. It has two ways of interacting with them.

**SQLAlchemy Core:** Uses SQL Expression Language that provides schema-centric usage paradigm
- **SQL Expression Language** allows you represent structures in a relational database using Python
- You can specify SQL statements in Python and use it directly
- This is the closest part to raw SQL in SQLAlchemy

**SQLAlchemy ORM:** Allows classes to be mapped to tables in the database to provide domain-centric usage.

## Resources

Quick Guide: https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_quick_guide.htm

## SQLAlchemy Core

### Connecting to Database

**Engine:** Class that provides a source of database connectivity and behaviour.

**MetaData:** A collection of *Table* objects and associated schema constructs
- Has an optional binding to an engine

In [2]:
# Define local sqlite database to be hosted in repo
# Different syntax for different database types
engine = create_engine('sqlite:///college.db', echo = True) # echo sets up SQLAlchemy logging - print SQL used
meta = MetaData()

# Define table
students = Table(
   'students', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String), 
)

# Use engine object to create table and other objects and store info in metadata
# This creates the local database
meta.create_all(engine)

2021-06-09 16:44:48,441 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-09 16:44:48,442 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2021-06-09 16:44:48,443 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-09 16:44:48,444 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("students")
2021-06-09 16:44:48,445 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-09 16:44:48,447 INFO sqlalchemy.engine.Engine 
CREATE TABLE students (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2021-06-09 16:44:48,448 INFO sqlalchemy.engine.Engine [no key 0.00069s] ()
2021-06-09 16:44:48,450 INFO sqlalchemy.engine.Engine COMMIT


**SQL expressions:** Constructed by using methods on tables
- Print corresponding SQL code using `str(<exp>)`

**Connection:** Represents an active *DBAPI* (Python Database API) connection
- Run expressions by passing to `execute(<exp>)` method on connection
- This also returns `ResultProxy` object that holds information

In [3]:
# Insert expression
ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')

# Equivalent SQL insert statement (without specific values)
print(str(ins))

# Specific values are stored in bind parameter which is visible in compiled form
print(ins.compile().params)

INSERT INTO students (name, lastname) VALUES (:name, :lastname)
{'name': 'Ravi', 'lastname': 'Kapoor'}


In [4]:
# Create connection to database
conn = engine.connect()
result = conn.execute(ins)

# Check last set of params inserted
print(result.last_inserted_params())

2021-06-09 16:44:48,478 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-06-09 16:44:48,481 INFO sqlalchemy.engine.Engine [generated in 0.00249s] ('Ravi', 'Kapoor')
2021-06-09 16:44:48,483 INFO sqlalchemy.engine.Engine COMMIT
{'name': 'Ravi', 'lastname': 'Kapoor'}


In [5]:
# Can insert many rows at once by passing list of dictionaries
conn.execute(students.insert(), [
   {'name':'Rajiv', 'lastname' : 'Khanna'},
   {'name':'Komal','lastname' : 'Bhandari'},
   {'name':'Abdul','lastname' : 'Sattar'},
   {'name':'Priya','lastname' : 'Rajhans'},
]);

2021-06-09 16:44:48,503 INFO sqlalchemy.engine.Engine INSERT INTO students (name, lastname) VALUES (?, ?)
2021-06-09 16:44:48,505 INFO sqlalchemy.engine.Engine [generated in 0.00198s] (('Rajiv', 'Khanna'), ('Komal', 'Bhandari'), ('Abdul', 'Sattar'), ('Priya', 'Rajhans'))
2021-06-09 16:44:48,507 INFO sqlalchemy.engine.Engine COMMIT


### Basic Select Statements

**Select:** Either a method on table or function with table as argument
- Limit columns by passing specific columns as argument of `select()` function
- Columns referenced using `<table>.c.<column>` (`c` is an alias for 'column')
- Rename columns using `.label()` method on columns in `select`

**ResultProxy:** Object returned by executing select
- Is iterable with items corresponding to rows
- Has other methods for accessing results: `fetchone()`, `fetchall()`,...

In [6]:
# Select all
s = students.select()
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

# Run expression
result = conn.execute(s)
print('')

for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 

2021-06-09 16:44:48,526 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2021-06-09 16:44:48,528 INFO sqlalchemy.engine.Engine [generated in 0.00190s] ()

(1, 'Ravi', 'Kapoor')
(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')
(4, 'Abdul', 'Sattar')
(5, 'Priya', 'Rajhans')


In [7]:
# Alternative syntax
s = select(students)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

# Run expression
result = conn.execute(s)
result.fetchall()

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 

2021-06-09 16:44:48,549 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students
2021-06-09 16:44:48,553 INFO sqlalchemy.engine.Engine [cached since 0.02705s ago] ()


[(1, 'Ravi', 'Kapoor'),
 (2, 'Rajiv', 'Khanna'),
 (3, 'Komal', 'Bhandari'),
 (4, 'Abdul', 'Sattar'),
 (5, 'Priya', 'Rajhans')]

In [8]:
# Specific columns
s = select(
    students.c.name, 
    students.c.lastname.label('surname')
)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

# Run expression
result = conn.execute(s)
result.fetchall()

Equivalent SQL
--------------
SELECT students.name, students.lastname AS surname 
FROM students 

2021-06-09 16:44:48,585 INFO sqlalchemy.engine.Engine SELECT students.name, students.lastname AS surname 
FROM students
2021-06-09 16:44:48,590 INFO sqlalchemy.engine.Engine [generated in 0.00481s] ()


[('Ravi', 'Kapoor'),
 ('Rajiv', 'Khanna'),
 ('Komal', 'Bhandari'),
 ('Abdul', 'Sattar'),
 ('Priya', 'Rajhans')]

### Where Clauses

**Where:** Use `.where()` method on `select` and pass column conditions
- Use `where(_and(<cond1>, <cond2>))` for multiple conditions
- Use `where(between(<col>, <val1>, <val2>)` for between

In [9]:
# Select first 3 rows
s = students.select().where(students.c.id <= 3)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

print('Params: ', s.compile().params, '\n')

result = conn.execute(s)
print('')
for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= :id_1 

Params:  {'id_1': 3} 

2021-06-09 16:44:48,612 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= ?
2021-06-09 16:44:48,614 INFO sqlalchemy.engine.Engine [generated in 0.00178s] (3,)

(1, 'Ravi', 'Kapoor')
(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')


In [10]:
# Select Ravi in first 3 rows
s = students.select().where(
    and_(
        students.c.id <= 3,
        students.c.name == 'Ravi'
    )
)
print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

print('Params: ', s.compile().params, '\n')

result = conn.execute(s)
print('')
for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= :id_1 AND students.name = :name_1 

Params:  {'id_1': 3, 'name_1': 'Ravi'} 

2021-06-09 16:44:48,629 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id <= ? AND students.name = ?
2021-06-09 16:44:48,633 INFO sqlalchemy.engine.Engine [generated in 0.00397s] (3, 'Ravi')

(1, 'Ravi', 'Kapoor')


In [11]:
# Select students with 2nd and 3rd ids
s = students.select().where(
    between(
        students.c.id,
        2,
        3
    )
)

print('Equivalent SQL')
print('--------------')
print(str(s), '\n')

print('Params: ', s.compile().params, '\n')

result = conn.execute(s)
print('')
for row in result:
    print(row)

Equivalent SQL
--------------
SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id BETWEEN :id_1 AND :id_2 

Params:  {'id_1': 2, 'id_2': 3} 

2021-06-09 16:44:48,662 INFO sqlalchemy.engine.Engine SELECT students.id, students.name, students.lastname 
FROM students 
WHERE students.id BETWEEN ? AND ?
2021-06-09 16:44:48,662 INFO sqlalchemy.engine.Engine [generated in 0.00095s] (2, 3)

(2, 'Rajiv', 'Khanna')
(3, 'Komal', 'Bhandari')


### Textual SQL

**TextClause:** Represents an SQL statement directly
- Constructed by passing SQL query into `text(<string>)`
- Parameters are denoted by colons and passed as kwargs into `execute()`
- Good when SQL is known and static

In [12]:
s = text('SELECT * FROM students WHERE students.id <= :x AND students.name = :y')
result = conn.execute(s, x=3, y='Ravi')
print('')

for row in result:
    print(row)

2021-06-09 16:44:48,679 INFO sqlalchemy.engine.Engine SELECT * FROM students WHERE students.id <= ? AND students.name = ?
2021-06-09 16:44:48,684 INFO sqlalchemy.engine.Engine [generated in 0.00491s] (3, 'Ravi')

(1, 'Ravi', 'Kapoor')


### Some Common Operations

**Alias:** Represents 'AS' in SQL statements
- There is a difference between the name of the variable representing alias table and the alias

**Order By:** Use `order_by(asc(<col>))` or `order_by(desc(<col>))` methods on select

**Functions:** Standard SQL functions are accessed through `func`
- Ex: `func.now()`, `func.count()`, `func.max()`

In [13]:
# Variable st represents students table aliased as 'a'
st = students.alias('a')
print('Original table name: ', students.name)
print('Alias table name: ', st.name)
print('')

# Run select all against aliased table
s = st.select().where(st.c.id == 1)
result = conn.execute(s)
result.fetchall()

Original table name:  students
Alias table name:  a

2021-06-09 16:44:48,706 INFO sqlalchemy.engine.Engine SELECT a.id, a.name, a.lastname 
FROM students AS a 
WHERE a.id = ?
2021-06-09 16:44:48,709 INFO sqlalchemy.engine.Engine [generated in 0.00305s] (1,)


[(1, 'Ravi', 'Kapoor')]

In [14]:
# Select students in alphabetical order
s = select(students).order_by(
    asc(students.c.name)
)
print(str(s))

SELECT students.id, students.name, students.lastname 
FROM students ORDER BY students.name ASC


In [15]:
# Count rows in table
s = select(func.count(students.c.id))
print(str(s))

SELECT count(students.id) AS count_1 
FROM students


### Joins

In [16]:
# Define new table
addresses = Table(
   'addresses', meta, 
   Column('id', Integer, primary_key = True), 
   Column('st_id', Integer, ForeignKey('students.id')), # Foreign key to students table
   Column('postal_add', String), 
   Column('email_add', String)
)

# Create the table
meta.create_all(engine)

# Populate new table
conn.execute(addresses.insert(), [
   {'st_id':1, 'postal_add':'Shivajinagar Pune', 'email_add':'ravi@gmail.com'},
   {'st_id':1, 'postal_add':'ChurchGate Mumbai', 'email_add':'kapoor@gmail.com'},
   {'st_id':3, 'postal_add':'Jubilee Hills Hyderabad', 'email_add':'komal@gmail.com'},
   {'st_id':5, 'postal_add':'MG Road Bangaluru', 'email_add':'as@yahoo.com'},
   {'st_id':2, 'postal_add':'Cannought Place new Delhi', 'email_add':'admin@khanna.com'},
]);

2021-06-09 16:44:48,766 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-09 16:44:48,769 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("students")
2021-06-09 16:44:48,772 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-09 16:44:48,774 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2021-06-09 16:44:48,775 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-09 16:44:48,778 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2021-06-09 16:44:48,779 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-09 16:44:48,781 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	st_id INTEGER, 
	postal_add VARCHAR, 
	email_add VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(st_id) REFERENCES students (id)
)


2021-06-09 16:44:48,782 INFO sqlalchemy.engine.Engine [no key 0.00124s] ()
2021-06-09 16:44:48,786 INFO sqlalchemy.engine.Engine COMMIT
2021-06-09 16:44:48,791 INFO sqlalchemy.engine.Engine INSERT INTO addresses (st_id, postal_

The simplest way to select from multiple tables is to pass both as arguments of `select` and define join condition in `where` clause

In [17]:
# Select all from both joining on foreign key
s = select(
    students,
    addresses
).where(
    students.c.id == addresses.c.st_id
)
print(str(s))

SELECT students.id, students.name, students.lastname, addresses.id AS id_1, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students, addresses 
WHERE students.id = addresses.st_id


**Join:** Method on tables
- Use `select_from()` method on `select` to explicitly set left hand side of join
- Can be multiple ways of phrasing
- Just specifying table name in `select` is equivalent to all columns from that table

In [18]:
# Join on its own
j = students.join(
    addresses,
    students.c.id == addresses.c.st_id,
    isouter = True # outer join - default is false
)
print(str(j))

students LEFT OUTER JOIN addresses ON students.id = addresses.st_id


In [19]:
# As part of select statement
s = select(
    students.c.name,
    students.c.lastname,
    addresses
).select_from(
    j # equivalent to block commented code below
    # students.join(
    #     addresses,
    #     students.c.id == addresses.c.st_id,
    #     isouter=True
    # )
)
print(str(s))

SELECT students.name, students.lastname, addresses.id, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students LEFT OUTER JOIN addresses ON students.id = addresses.st_id


In [20]:
# An equivalent formulation
s = select(
    students.c.name,
    students.c.lastname,
    addresses
).select_from(
    students
).join(
    addresses,
    students.c.id == addresses.c.st_id,
    isouter=True
)
print(str(s))

SELECT students.name, students.lastname, addresses.id, addresses.st_id, addresses.postal_add, addresses.email_add 
FROM students LEFT OUTER JOIN addresses ON students.id = addresses.st_id


## Reset

In [21]:
# # Remove database to reset everything
# engine.dispose()
# os.remove('college.db')